In [ ]:
!pip install tqdm pandas scikit-learn openpyxl 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from dataclasses import make_dataclass
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
INPUT_DATA = "/content/drive/MyDrive/Colab Notebooks/hyperion/ml/retete_output_scores.xlsx"
OUTPUT_DATA_ML = "/content/drive/MyDrive/Colab Notebooks/hyperion/ml/output_data.xlsx"
OUTPUT_DATA_ML_TEST = "/content/drive/MyDrive/Colab Notebooks/hyperion/ml/output_data_test.xlsx"

In [ ]:
def to_binary(value, number_of_bits, lsb=False):
    """
    Function return two's complement representation
    :param value: value in decimal representation
    :param number_of_bits: number of bits used for representation
    :returns: np.array that represents the binary representation
    >>> to_binary(10,4)
    array([1, 0, 1, 0])
    >>> to_binary(10,4,True)
    array([0, 1, 0, 1])
    """
    if lsb == True:
        return np.flip(np.array(list(np.binary_repr(value, number_of_bits)), dtype=int))
    return np.array(list(np.binary_repr(value, number_of_bits)), dtype=int)


In [ ]:
def get_interaction_pairs_from_list(drug_list):
    interaction_pairs = []
    for drug1 in drug_list:
        for drug2 in drug_list:
            if drug1 == drug2:
                continue
            elif (drug1, drug2) in interaction_pairs or (drug2, drug1) in interaction_pairs:
                continue
            else:
                interaction_pairs.append((drug1, drug2))
    return interaction_pairs

In [ ]:
df = pd.read_excel(INPUT_DATA, sheet_name=None)
df_cardio = df['retete cardiovascuar']

In [ ]:
EntryDec = make_dataclass("Entry", [("nr",str),
                                ("d1",str),
                                 ("d2",str),
                                 ("diag",int),
                                 ("sex",int),
                                 ("age",int),
                                 ("diag_type",int),
                                 ("s1",int),
                                 ("s2",int),
                                 ("bpfp",int)
                                 
                                 ])

entries_list = []
entries_list_dec = []

for index, row in tqdm(df_cardio.iterrows(), total=df_cardio.shape[0]):
    comment = row['comments']
    if pd.isna(comment):
        nr_rp = row['nr rp']
        db_ids = list(filter(None, re.split("[, \-!?:]+", row['DB ID'])))
        pairs = get_interaction_pairs_from_list(db_ids)
       
        diagnostic = list(filter(None, re.split("[, \-!?:]+", str(row['cod diagnostic'])))) 
        sex = 1 if row['sex'] is 'M' else 0 if row['sex'] is 'F' else 1
        age = int(row['varsta']) if not pd.isna(row['varsta']) else 20
        diag_type = 1 if row['tip diagnostic'] is 'cr' else 0
        s1 = int(row['scor numar interactiuni'])
        s2 = int(row['eliminare'])
        bpfp = int(row['B,P,FP'])
        
        diag = np.zeros((1000,),dtype=int)
       
        # print(diagnostic)
        for d in diagnostic:
            diag[int(float(d))-1]=1
        diag_value = 0
        for i in range(0,1000):
            if diag[i]==1:
                diag_value+=2^i
        
        for (d1,d2) in pairs:
            entry_dec = EntryDec(str(nr_rp),
                d1,
                d2,
                diag_value,
                sex,
                age,
                diag_type,
                s1,
                s2,
                bpfp
            )
            # entries_list.append(entry)
            entries_list_dec.append(entry_dec)
out_df=pd.DataFrame(entries_list_dec)
        


# # Interaction pair
# # diag code de la 1 la 1000 one hot
# # sex - 0 = male, 1 = female
# # age => binary
# # type => acut, subacut 0, cronic = 1
# # scor interactiuni -> binary
# # eliminare -> binary
# # bpfp -> b=0,p=1,fp=2

100%|██████████| 1432/1432 [00:00<00:00, 3173.97it/s]


In [ ]:
from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()
out_df

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
with pd.ExcelWriter(OUTPUT_DATA_ML) as writer:
    out_df.to_excel(writer, sheet_name="Sheet_1")

In [ ]:
df = pd.read_excel(INPUT_DATA, sheet_name=None)
df_cardio = df['retete cardiovascuar']

In [ ]:

EntryTest = make_dataclass("Entry", [("nr",str),
                                 ("d1",str),
                                 ("d2",str),
                                 ("diag",int),
                                 ("sex",int),
                                 ("age",int),
                                 ("diag_type",int),
                                 ("s1",int),
                                 ("s2",int),
                  
                                 
                                 ])

entries_test=[]
for index, row in tqdm(df_cardio.iterrows(), total=df_cardio.shape[0]):
    comment = row['comments']
    if not pd.isna(comment):
        nr_rp = row['nr rp']
        db_ids = list(filter(None, re.split("[, \-!?:]+", row['DB ID'])))
        pairs = get_interaction_pairs_from_list(db_ids)
       
        diagnostic = list(filter(None, re.split("[, \-!?:]+", str(row['cod diagnostic'])))) 
        sex = 1 if row['sex'] is 'M' else 0 if row['sex'] is 'F' else 1
        age = int(row['varsta']) if not pd.isna(row['varsta']) else 20
        diag_type = 1 if row['tip diagnostic'] is 'cr' else 0
        s1 = int(row['scor numar interactiuni'])
        s2 = int(row['eliminare'])
       
        
        diag = np.zeros((1000,),dtype=int)
       
        print(diagnostic)
        for d in diagnostic:
            diag[int(float(d))-1]=1
        diag_value = 0
        for i in range(0,1000):
            if diag[i]==1:
                diag_value+=2^i
        
        entry_dec = EntryTest(str(nr_rp),
                d1,
                d2,
                diag_value,
                sex,
                age,
                diag_type,
                s1,
                s2,
                
            )
            
        entries_test.append(entry_dec)
test_df=pd.DataFrame(entries_test)
        


# Interaction pair
# diag code de la 1 la 1000 one hot
# sex - 0 = male, 1 = female
# age => binary
# type => acut, subacut 0, cronic = 1
# scor interactiuni -> binary
# eliminare -> binary
# bpfp -> b=0,p=1,fp=2

  5%|▌         | 74/1432 [00:00<00:03, 385.53it/s]

['289', '453', '653']
['453', '290', '648', '559', '289', '476']
['453', '632', '481']
['701', '289', '632', '573', '350', '481', '453']
['453', '462']
['462', '453']
['462', '526', '453']
['453', '462', '701']
['453', '497', '559']
['462', '453']
['462', '289', '453']
['462', '453']
['453', '462']
['453', '487', '470']
['289', '453', '471']
['289', '470', '453', '487']
['453', '476', '462', '289', '491']
['453', '476', '462', '289']
['453', '476', '289', '461']
['453', '476', '289', '461']
['453', '486']
['497', '453', '289', '239']
['453', '476', '462', '289', '491']
['453', '476', '462', '289']
['453', '476', '462', '289', '491']
['497', '453', '559', '462']
['453', '462', '497']
['289', '290', '453', '497', '559', '701']
['453', '462', '476', '476']
['290', '482', '476', '453', '289']
['476', '559', '453', '289']
['453', '497', '289']
['453', '289']
['462', '453', '290', '289']
['497', '453', '559', '651', '289', '481']
['453', '462', '481', '701']
['476', '453', '289', '350']
['45

  8%|▊         | 113/1432 [00:00<00:03, 361.88it/s]


['376', '377', '453']
['453', '377', '376']
['376', '377', '453']
['453', '481', '415']
['453', '289', '481']
['453', '701', '438']
['298', '453', '481']
['453', '289', '481']
['453', '289', '481']
['453', '606', '481', '514']
['453.0']
['453', '289', '438']
['453', '289', '559', '481']
['453', '648', '481', '491']
['290', '289', '526', '453', '438']
['453', '289', '701', '481']
['453', '289', '481', '415']
['289', '481', '453']
['453', '630', '481', '289']
['462', '453', '481', '632', '559']
['632', '462', '453', '438']
['462', '325', '289', '453', '701', '481']
['458', '453', '289', '438']
['290', '482', '453', '289', '376']
['453', '289', '481']
['630', '453', '701', '383', '289', '481']
['289', '453', '481', '491']
['462', '559', '453', '500', '438', '481', '289']
['453', '462', '481', '559']
['453', '290', '559', '527', '481']
['453', '481', '289', '497', '373']
['140', '453', '289', '321']
['453', '289', '438', '527']
['100', '453', '377']
['124', '453', '462', '289']
['453', '2

 13%|█▎        | 184/1432 [00:00<00:04, 284.93it/s]

...

 39%|███▉      | 564/1432 [00:01<00:02, 409.68it/s]


['453', '463']
['453', '485', '289']
['453', '485', '289']
['524', '453', '462', '701']
['559', '368', '453', '701', '368']
['453', '289', '462']
['289', '630', '453']
['453', '573', '289', '651', '559', '648']
['453', '289', '573']
['701', '453', '573']
['453', '514', '573']
['462', '289', '453', '573']
['341', '415', '453', '553', '701']
['573', '453', '289', '651', '559']
['453', '462']
['453', '462']
['453', '462']
['462', '453']
['453', '454']
['462', '453']
['453', '461', '462', '630', '701']
['117', '453', '462']
['124', '453', '289']
['124', '453', '462', '289']
['559', '453', '289', '701']
['289', '453', '454']
['453', '648', '481']
['453', '462', '568', '632', '559', '632', '481']
['453', '527', '653']
['453', '462', '481']
['453', '462', '481']
['633', '453', '289', '462', '396']
['453', '462', '481', '603']
['453', '603', '462', '481']
['111', '453', '462', '701']
['453', '462', '523', '289', '487']
['453', '289', '462']
['453', '462']
['453', '289', '462']
['453', '289', 

 45%|████▌     | 649/1432 [00:01<00:01, 417.17it/s]


['453', '462', '290', '289']
['453', '454', '460']
['289', '290', '425', '453', '458', '497']
['137', '236', '453', '290', '454']
['289', '694', '453']
['453', '289', '521', '462']
['140', '453', '289', '701', '239']
['453', '559', '289', '481']
['462', '453', '289', '491', '648', '559']
['462', '453', '289', '559']
['462', '453', '289', '559']
['453', '462', '479']
['438', '453']
['289', '453', '350', '701', '573']
['289', '453', '350', '701']
['453', '695', '289', '481']
['453', '695', '289', '481']
['453', '289', '350']
['453', '462', '479']
['453', '481', '462']
['453', '350', '237']
['453', '350', '237']
['289', '453', '481']
['453', '491', '481']
['462', '453', '479']
['453', '289', '661', '480']
['383', '453', '491']
['453', '661', '481']
['289', '350', '453', '462', '364', '368']
['289', '368', '453', '462', '481', '556']
['462', '453', '350', '526', '289']
['98', '453']
['453.0']
['453.0']
['453.0']
['453.0']
['453.0']
['453.0']
['453.0']
['289', '453']
['453.0']
['453', '289

100%|██████████| 1432/1432 [00:01<00:00, 746.69it/s] 


In [ ]:
from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()
test_df

,nr,d1,d2,diag,sex,age,diag_type,s1,s2
0,3120.0,DB00808,DB00316,1398,1,64,0,1000,2
1,1327.0,DB00808,DB00316,2709,1,63,0,400,0
2,3049.0,DB00808,DB00316,1565,0,93,0,200,0
3,3053.0,DB00808,DB00316,3482,1,87,0,0,0
4,2875.0,DB00808,DB00316,917,1,89,0,0,0
...,...,...,...,...,...,...,...,...,...
670,663.0,DB00808,DB00316,2763,1,65,0,1800,2
671,704.0,DB00808,DB00316,2766,1,66,0,1800,2
672,1162.0,DB00808,DB00316,2766,1,66,0,1800,2
673,1416.0,DB00808,DB00316,744,1,77,0,400,0


In [ ]:
with pd.ExcelWriter(OUTPUT_DATA_ML_TEST) as writer:
    test_df.to_excel(writer, sheet_name="Sheet_1")